# 🧠 AI‑Powered HR Assistant — Nestlé HR Policy (Course‑End Project)

This notebook demonstrates an end‑to‑end workflow to build a **conversational RAG chatbot** that answers queries about **Nestlé’s HR policy PDF** using:
- **LangChain** (loading, splitting, retrieval pipeline)
- **ChromaDB** (vector database)
- **OpenAI** (embeddings + GPT chat model)
- **Gradio** (chat UI)

> **What you'll do:** set up the environment, load & chunk the PDF, create embeddings and a vector store, retrieve context, and use GPT‑3.5‑Turbo to answer questions. Finally, you'll launch a Gradio chat app.

In [ ]:
# If running locally, uncomment to install dependencies.
# Note: You don't need to run this cell in the LMS if the environment already has them.
# !pip -q install --upgrade pip
# !pip -q install "langchain>=0.2.6" "langchain-community>=0.2.6" "langchain-openai>=0.1.8" \    #                 "chromadb>=0.5.3" "pypdf>=4.2.0" "tiktoken>=0.7.0" "gradio>=4.35.0" "python-dotenv>=1.0.1"

In [ ]:
import os
from typing import List

# Load .env if present (optional)
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass

# === IMPORTANT ===
# Set your OpenAI API key securely (prefer .env file with OPENAI_API_KEY=...)
# or set it in the environment before running the notebook.
# os.environ["OPENAI_API_KEY"] = "sk-..."
assert os.getenv("OPENAI_API_KEY"), "Please set OPENAI_API_KEY in your environment or a .env file."

# LangChain / OpenAI wrappers
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# LCEL building blocks
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# UI
import gradio as gr

In [ ]:
# ---------------- Configuration ----------------
PDF_PATH = "data/nestle_hr_policy.pdf"      # <-- Put your Nestlé HR policy PDF here
PERSIST_DIR = "chroma_db_nestle_hr"         # persistent folder for Chroma
EMBED_MODEL = "text-embedding-3-small"      # cost-effective; can use 'text-embedding-3-large' for higher quality
CHAT_MODEL = "gpt-3.5-turbo"                # as per assignment; you may upgrade later (e.g., 'gpt-4o-mini')

os.makedirs("data", exist_ok=True)
print(f"Expecting policy at: {PDF_PATH}")

In [ ]:
# -------- Load PDF --------
loader = PyPDFLoader(PDF_PATH)
docs = loader.load()
print(f"Loaded {len(docs)} pages.")

# -------- Split into chunks --------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=150, separators=["\n\n", "\n", " ", ""],
)
chunks = text_splitter.split_documents(docs)
print(f"Created {len(chunks)} chunks.")

In [ ]:
# -------- Create embeddings + VectorStore (Chroma) --------
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=PERSIST_DIR,
)
vectordb.persist()
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
print("Vector store built & persisted.")

In [ ]:
# -------- Prompt template for grounded answers --------
system_msg = (
    "You are a helpful HR assistant for Nestlé. "
    "Answer ONLY using the provided context. "
    "If the answer is not in the context, say you cannot find it in the policy. "
    "Be concise and use bullet points when listing items. "
    "Do NOT fabricate citations; page references will be appended automatically."
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_msg),
        ("human", "Question: {question}\n\nContext:\n{context}\n\nHelpful answer:"),
    ]
)

llm = ChatOpenAI(model=CHAT_MODEL, temperature=0)
parser = StrOutputParser()

def _format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# LCEL RAG chain: {question} -> retrieve -> prompt -> LLM -> text
rag_chain = (
    {
        "context": retriever | _format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | parser
)

# Small helper to create page citations
def pages_citation(docs):
    pages = sorted({(d.metadata.get("page", None) or 0) + 1 for d in docs})
    if pages:
        return "Sources: " + ", ".join(f"p.{p}" for p in pages)
    return "Sources: (none)"

# Test a dry run (uncomment after you place the PDF):
# q = "What is the probation period policy for new hires?"
# docs_for_q = retriever.get_relevant_documents(q)
# print(rag_chain.invoke(q) + "\n\n" + pages_citation(docs_for_q))

In [ ]:
# -------- Gradio Chat Interface --------
EXAMPLE_QUESTIONS = [
    "What are the working hours and overtime rules?",
    "What is the leave policy (annual, sick, parental)?",
    "How does the performance review process work?",
    "Who do I contact for benefits enrollment?",
    "What is the probation period for new employees?",
]

# Chat function compatible with gr.ChatInterface
def chat_fn(message, history):
    try:
        # Retrieve docs for explicit citation
        docs = retriever.get_relevant_documents(message)
        answer = rag_chain.invoke(message)
        return answer + "\n\n" + pages_citation(docs)
    except Exception as e:
        return f"⚠️ Error: {e}"

demo = gr.ChatInterface(
    fn=chat_fn,
    title="Nestlé HR Policy Assistant",
    description=(
        "Ask questions about Nestlé’s HR policy. "
        "Answers are grounded in the uploaded PDF. "
        "We'll append the page sources below each answer."
    ),
    examples=EXAMPLE_QUESTIONS,
    textbox=gr.Textbox(placeholder="Type your HR question…", container=True, scale=7),
    retry_btn="Retry",
    undo_btn=None,
    clear_btn="Clear",
)

# To run locally: uncomment the next line.
# demo.launch()

## 🔎 (Bonus) Query Structuring
In some cases, you may wish to **rewrite** the user's question into a clearer, more retrieval‑friendly form.
The snippet below shows how to build a lightweight "query rewriter" using the same chat model.

In [ ]:
from langchain_core.prompts import PromptTemplate

rewrite_template = PromptTemplate.from_template(
    "Rewrite the user question into a short, clear search query for HR policy retrieval.\n"
    "User question: {q}\n"
    "Rewritten search query:"
)

def rewrite_query(q: str) -> str:
    rewritten = (rewrite_template | llm | parser).invoke({"q": q}).strip()
    return rewritten or q

# Example:
# rewrite_query("Could you explain the rules for taking vacation and sick leave?")

In [ ]:
def ask_hr(question: str, use_rewriter: bool = False) -> str:
    q = rewrite_query(question) if use_rewriter else question
    docs = retriever.get_relevant_documents(q)
    answer = rag_chain.invoke(q)
    return answer + "\n\n" + pages_citation(docs)

# Example:
# ask_hr("What are the maternity leave benefits?", use_rewriter=True)

## ✅ Result
You now have a working **RAG chatbot** for Nestlé’s HR policy with:
- PDF ingestion & chunking
- Embeddings & vector search (Chroma)
- GPT‑3.5‑Turbo question answering grounded in retrieved context
- A Gradio chat interface with page citations

### Submission tips
- Ensure your **`data/nestle_hr_policy.pdf`** file exists before running.
- Run all cells end‑to‑end and include relevant screenshots in your LMS submission.
- You can export the notebook to HTML (`File → Save and Export Notebook As… → HTML`) if required.